In [1]:
from sklearn import tree
import pandas as pd
import os

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df[['koi_steff', 'koi_slogg', 'koi_srad']] # 'koi_smet', 'koi_smass', 'koi_sage']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


label_encoder.fit(df['koi_disposition'])
label_encoder.classes_

array(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

In [5]:
y_values = label_encoder.transform(df['koi_disposition'])#.astype('int')
y = y_values.reshape(-1, 1)
type(y)

numpy.ndarray

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
X_train.head()

koi_steff  koi_slogg  koi_srad
6122       5737      4.327     1.125
6370       5855      4.578     0.797
2879       6328      4.481     0.963
107        4768      4.536     0.779
29         5712      4.359     1.082

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

print(f"Y Shape: {y_train.shape}")
print(f"X Shape: {X_train_scaled.shape}")

Y Shape: (5243, 1)
X Shape: (5243, 3)


# Train the Model



In [9]:
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)
model.score(X_test_scaled, y_test)

0.5045766590389016

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [10]:
model.get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}

grid = GridSearchCV(model, param_grid, verbose=3)

In [12]:
grid.fit(X_train_scaled, y_train.ravel())

Fitting 5 folds for each of 294 candidates, totalling 1470 fits
[CV 1/5] END ..........max_leaf_nodes=2, min_samples_split=2; total time=   0.0s
[CV 2/5] END ..........max_leaf_nodes=2, min_samples_split=2; total time=   0.0s
[CV 3/5] END ..........max_leaf_nodes=2, min_samples_split=2; total time=   0.0s
[CV 4/5] END ..........max_leaf_nodes=2, min_samples_split=2; total time=   0.0s
[CV 5/5] END ..........max_leaf_nodes=2, min_samples_split=2; total time=   0.0s
[CV 1/5] END ..........max_leaf_nodes=2, min_samples_split=3; total time=   0.0s
[CV 2/5] END ..........max_leaf_nodes=2, min_samples_split=3; total time=   0.0s
[CV 3/5] END ..........max_leaf_nodes=2, min_samples_split=3; total time=   0.0s
[CV 4/5] END ..........max_leaf_nodes=2, min_samples_split=3; total time=   0.0s
[CV 5/5] END ..........max_leaf_nodes=2, min_samples_split=3; total time=   0.0s
[CV 1/5] END ..........max_leaf_nodes=2, min_samples_split=4; total time=   0.0s
[CV 2/5] END ..........max_leaf_nodes=2, min_

[CV 4/5] END ..........max_leaf_nodes=9, min_samples_split=3; total time=   0.0s
[CV 5/5] END ..........max_leaf_nodes=9, min_samples_split=3; total time=   0.0s
[CV 1/5] END ..........max_leaf_nodes=9, min_samples_split=4; total time=   0.0s
[CV 2/5] END ..........max_leaf_nodes=9, min_samples_split=4; total time=   0.0s
[CV 3/5] END ..........max_leaf_nodes=9, min_samples_split=4; total time=   0.0s
[CV 4/5] END ..........max_leaf_nodes=9, min_samples_split=4; total time=   0.0s
[CV 5/5] END ..........max_leaf_nodes=9, min_samples_split=4; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=10, min_samples_split=2; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=10, min_samples_split=2; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=10, min_samples_split=2; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=10, min_samples_split=2; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=10, min_samples_split=2; total time=   0.0s
[CV 1/5] END .........max_le

[CV 2/5] END .........max_leaf_nodes=17, min_samples_split=2; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=17, min_samples_split=2; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=17, min_samples_split=2; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=17, min_samples_split=2; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=17, min_samples_split=3; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=17, min_samples_split=3; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=17, min_samples_split=3; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=17, min_samples_split=3; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=17, min_samples_split=3; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=17, min_samples_split=4; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=17, min_samples_split=4; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=17, min_samples_split=4; total time=   0.0s
[CV 4/5] END .........max_le

[CV 2/5] END .........max_leaf_nodes=24, min_samples_split=2; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=24, min_samples_split=2; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=24, min_samples_split=2; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=24, min_samples_split=2; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=24, min_samples_split=3; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=24, min_samples_split=3; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=24, min_samples_split=3; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=24, min_samples_split=3; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=24, min_samples_split=3; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=24, min_samples_split=4; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=24, min_samples_split=4; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=24, min_samples_split=4; total time=   0.0s
[CV 4/5] END .........max_le

[CV 2/5] END .........max_leaf_nodes=32, min_samples_split=3; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=32, min_samples_split=3; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=32, min_samples_split=3; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=32, min_samples_split=3; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=32, min_samples_split=4; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=32, min_samples_split=4; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=32, min_samples_split=4; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=32, min_samples_split=4; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=32, min_samples_split=4; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=33, min_samples_split=2; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=33, min_samples_split=2; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=33, min_samples_split=2; total time=   0.0s
[CV 4/5] END .........max_le

...

[CV 4/5] END .........max_leaf_nodes=78, min_samples_split=3; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=78, min_samples_split=3; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=78, min_samples_split=4; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=78, min_samples_split=4; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=78, min_samples_split=4; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=78, min_samples_split=4; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=78, min_samples_split=4; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=79, min_samples_split=2; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=79, min_samples_split=2; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=79, min_samples_split=2; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=79, min_samples_split=2; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=79, min_samples_split=2; total time=   0.0s
[CV 1/5] END .........max_le

[CV 4/5] END .........max_leaf_nodes=85, min_samples_split=4; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=85, min_samples_split=4; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=86, min_samples_split=2; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=86, min_samples_split=2; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=86, min_samples_split=2; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=86, min_samples_split=2; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=86, min_samples_split=2; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=86, min_samples_split=3; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=86, min_samples_split=3; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=86, min_samples_split=3; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=86, min_samples_split=3; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=86, min_samples_split=3; total time=   0.0s
[CV 1/5] END .........max_le

[CV 5/5] END .........max_leaf_nodes=92, min_samples_split=4; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=93, min_samples_split=2; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=93, min_samples_split=2; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=93, min_samples_split=2; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=93, min_samples_split=2; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=93, min_samples_split=2; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=93, min_samples_split=3; total time=   0.0s
[CV 2/5] END .........max_leaf_nodes=93, min_samples_split=3; total time=   0.0s
[CV 3/5] END .........max_leaf_nodes=93, min_samples_split=3; total time=   0.0s
[CV 4/5] END .........max_leaf_nodes=93, min_samples_split=3; total time=   0.0s
[CV 5/5] END .........max_leaf_nodes=93, min_samples_split=3; total time=   0.0s
[CV 1/5] END .........max_leaf_nodes=93, min_samples_split=4; total time=   0.0s
[CV 2/5] END .........max_le

[CV 5/5] END .........max_leaf_nodes=99, min_samples_split=4; total time=   0.0s


GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=3)

In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'max_leaf_nodes': 90, 'min_samples_split': 2}
0.5525453176052804


# Save the Model

In [ ]:
# import joblib
# filename = 'LogisticRegression.sav'
# joblib.dump(grid, filename)